# Ejercicio 7: Bases de Datos Vectoriales

## Objetivo de la práctica

Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales

## Parte 0: Carga del Corpus

Vamos a utilizar la API de Kaggle para acceder al dataset _Wikipedia Text Corpus for NLP and LLM Projects_

El corpus está disponible desde este [link](https://www.kaggle.com/datasets/gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects?utm_source=chatgpt.com)

### Actividad

1. Carga el corpus


In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [2]:
# Set the path to the file you'd like to load
file_path = "wikipedia_text_corpus.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
)

df.head()

,Unnamed: 0,text
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...
1,2,Battery indicator\n\nA battery indicator (also...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...


## Parte 1: Generación de Embeddings

Vamos a utilizar E5 como modelo de embeddings.

La documentación de E5 está disponible desde este [link](https://huggingface.co/intfloat/e5-base-v2)

### Actividad

1. Normalizar el corpus
2. Definir una función `chunk_text`, y dividir los textos en _chunks_.
3. Generar embeddings por cada _chunk_

In [3]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["text"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

,Unnamed: 0,text,text_norm
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...,Anovo Anovo (formerly A Novo) is a computer se...
1,2,Battery indicator\n\nA battery indicator (also...,Battery indicator A battery indicator (also kn...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19...","Bob Pease Robert Allen Pease (August 22, 1940Â..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...,CAVNET CAVNET was a secure military forum whic...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...,CLidar The CLidar is a scientific instrument u...


In [5]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

(   doc_id  chunk_id                                               text
 0       0         0  Anovo Anovo (formerly A Novo) is a computer se...
 1       1         0  Battery indicator A battery indicator (also kn...
 2       1         1  ad battery when in reality it indicates a prob...
 3       1         2  s that an internal standby battery needs repla...
 4       1         3  increase; in many cases the EMF remains more o...,
 79104)

In [6]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

# Embeddings (N x D)
# normalize_embeddings=True es útil si usarás cosine similarity
embeddings = model.encode(
    passages,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

embeddings.shape, embeddings.dtype

2026-01-06 02:45:41.357679: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767667541.830564      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767667541.938407      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767667543.054658      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767667543.054701      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767667543.054704      55 computation_placer.cc:177] computation placer alr

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Batches:   0%|          | 0/1236 [00:00<?, ?it/s]

((79104, 768), dtype('float32'))

In [7]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_vec = embed_query(query_text)
query_vec.shape

(1, 768)

## Parte 2: FAISS

FAISS es una librería para búsqueda por similitud eficiente y clustering de vectores densos.

La documentación de FAISS está disponible en este [link](https://faiss.ai/index.html)

### Actividad

1. Crea un índice en FAISS
2. Carga los embeddings
3. Realiza una búsqueda a partir de una _query_

In [8]:
# código base para FAISS
import faiss
import numpy as np

# Asumiendo `embeddings` en un array NxD
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

D, I = index.search(query_embedding, k=10)

ModuleNotFoundError: No module named 'faiss'

## Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)

### Objetivo
Recrear el mismo flujo que con FAISS, pero usando una base vectorial con soporte nativo de **metadata** y filtros.

### Qué debes implementar
1. Levantar / conectar con una instancia de Qdrant.
2. Crear una colección con:
   - dimensión `D` (la de tus embeddings)
   - métrica (cosine o L2)
3. Insertar:
   - `id`
   - `embedding`
   - `payload` (metadata: texto, título, etiquetas, etc.)
4. Consultar Top-k por similitud:
   - `query_embedding`
   - `k`

### Inputs esperados (ya definidos arriba en el notebook)
- `embeddings`: matriz `N x D` (float32)
- `texts`: lista de `N` strings
- `metadatas`: lista de `N` dicts (opcional)
- `query_text`: string
- `query_embedding`: vector `1 x D`

### Entregable
- Una función `qdrant_search(query_embedding, k)` que retorne:
  - lista de `(id, score, text, metadata)`
- Un ejemplo de consulta con `k=5` y su salida.

### Preguntas
- ¿La métrica usada fue cosine o L2? ¿Por qué?
- ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
- ¿Qué pasa con el tiempo de respuesta cuando aumentas `k`?


In [9]:
!pip install qdrant-client pymilvus weaviate-client chromadb psycopg2-binary pgvector

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 8.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.1/285.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.7/603.7 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 94.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 92.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 106.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

# subir a memoria
client_qdrant = QdrantClient(":memory:")

#  crear colección
collection_name = "wikipedia_chunks"
vector_size = embeddings.shape[1]

# Verificamos si existe y la borramos para empezar de cero
if client_qdrant.collection_exists(collection_name):
    client_qdrant.delete_collection(collection_name)

client_qdrant.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
)

# Preparar los puntos para insertar
points = []
for i, row in chunks_df.iterrows():
    points.append(PointStruct(
        id=i,  # Usamos el índice como ID
        vector=embeddings[i].tolist(),
        payload={"text": row["text"], "doc_id": row["doc_id"], "chunk_id": row["chunk_id"]}
    ))

# Insertar 
client_qdrant.upsert(
    collection_name=collection_name,
    points=points
)
print("Datos cargados correctamente en Qdrant.")

# Función de búsqueda 
def qdrant_search(query_vec, k=5):
    search_result = client_qdrant.query_points(
        collection_name=collection_name,
        query=query_vec.flatten().tolist(), 
        limit=k
    ).points # query_points devuelve un objeto QueryResponses
    
    results = []
    for hit in search_result:
        results.append({
            "id": hit.id,
            "score": hit.score,
            "text": hit.payload["text"],
            "metadata": {"doc_id": hit.payload["doc_id"]}
        })
    return results

# Prueba
print("     Resultados Qdrant")
results_qdrant = qdrant_search(query_vec, k=5)
for res in results_qdrant:
    print(f"ID: {res['id']}, Score: {res['score']:.4f}\nTexto: {res['text'][:100]}...\n")

/tmp/ipykernel_55/1303288031.py:30: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 79104 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  client_qdrant.upsert(


Datos cargados correctamente en Qdrant.
     Resultados Qdrant
ID: 10176, Score: 0.8703
Texto: Battery tester A battery tester is an electronic device intended for testing the state of an electri...

ID: 1, Score: 0.8618
Texto: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives inform...

ID: 10177, Score: 0.8401
Texto: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-ac...

ID: 37406, Score: 0.8391
Texto: ils. One was connected via a series resistor to the battery supply. The second was connected to the ...

ID: 71872, Score: 0.8386
Texto: is achieved. Accepted average float voltages for lead-acid batteries at 25 Â°C can be found in follo...



1. ¿La métrica usada fue cosine o L2? ¿Por qué?
Se utilizó la Similitud del Coseno y laa razón técnica principal es que el modelo utilizado para generar los vectores (E5) fue entrenado para medir la similitud semántica basándose en la dirección y el ángulo entre los vectores, no en la distancia física entre ellos. Aunque al normalizar los datos la distancia Euclídea (L2) y el Coseno son muy similares, el Coseno sigue siendo el estándar más adecuado para tareas de procesamiento de texto.

2. ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
El proceso en Qdrant fue considerablemente más sencillo y eficiente que en FAISS. Mientras que FAISS se centra exclusivamente en la indexación de vectores y suele requerir un sistema externo para gestionar los datos asociados, Qdrant permite almacenar los metadatos junto con el vector. Esto permite realizar la búsqueda y el filtrado en una única operación integrada, eliminando la necesidad de gestionar índices paralelos.

3. ¿Qué pasa con el tiempo de respuesta cuando aumentas k?
Al aumentar el valor de k que era el número de resultados solicitados, el tiempo de respuesta sufre un incremento mínimo, que resulta imperceptible para el usuario en este volumen de datos. Esto ocurre porque el costo computacional principal esta en localizar la región cercana en el espacio vectorial, una vez localizada, recuperar 5 o 20 documentos adicionales requiere un esfuerzo de procesamiento muy bajo para el motor de búsqueda.

## Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

### Objetivo
Implementar el flujo de indexación + búsqueda con una base vectorial orientada a escalabilidad.

### Qué debes implementar
1. Conectar a Milvus.
2. Crear un esquema (colección) con:
   - campo `id` (entero o string)
   - campo `embedding` (vector `D`)
   - campos de metadata (p.ej., `category`, `source`, `title`)
3. Insertar `N` embeddings.
4. Crear/seleccionar un índice ANN (ej. HNSW o IVF).
5. Ejecutar consultas Top-k y recuperar textos asociados.

### Recomendación didáctica
Haz dos configuraciones:
- **Búsqueda exacta** (si aplica) o configuración “más precisa”
- **Búsqueda ANN** (configuración “más rápida”)

Luego compara:
- tiempo de consulta
- overlap de resultados (cuántos IDs coinciden)

### Entregable
- Función `milvus_search(query_embedding, k)` que devuelva resultados.
- Un mini experimento: `k=5` y `k=20` (tiempos y resultados).

### Preguntas
- ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?
- ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?


In [12]:
# Instalar la versión ligera de Milvus para uso local
!pip install "pymilvus[milvus-lite]"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 35.0 MB/s eta 0:00:00:00:0100:01


In [13]:
from pymilvus import MilvusClient
from tqdm.auto import tqdm 

client_milvus = MilvusClient("milvus_demo.db")

# Crear colección
collection_name = "wiki_milvus"

if client_milvus.has_collection(collection_name):
    client_milvus.drop_collection(collection_name)

client_milvus.create_collection(
    collection_name=collection_name,
    dimension=embeddings.shape[1],
    metric_type="COSINE", 
    auto_id=False 
)

# Preparar datos
print("Preparando datos para Milvus...")
data_milvus = []
for i, row in chunks_df.iterrows():
    data_milvus.append({
        "id": i,
        "vector": embeddings[i].tolist(),
        "text": row["text"],
        "doc_id": row["doc_id"]
    })

# Insertar datos por lotes
batch_size = 5000 
print(f"Insertando {len(data_milvus)} registros en lotes de {batch_size}...")

for i in tqdm(range(0, len(data_milvus), batch_size)):
    batch = data_milvus[i : i + batch_size]
    client_milvus.insert(collection_name=collection_name, data=batch)

print("insercion completada ")

# Función de búsqueda
def milvus_search(query_vec, k=5):
    search_res = client_milvus.search(
        collection_name=collection_name,
        data=[query_vec.flatten().tolist()],
        limit=k,
        search_params={"metric_type": "COSINE", "params": {}}, 
        output_fields=["text", "doc_id"]
    )
    
    formatted_results = []
    for hit in search_res[0]:
        formatted_results.append({
            "id": hit["id"],
            "score": hit["distance"],
            "text": hit["entity"]["text"],
            "metadata": {"doc_id": hit["entity"]["doc_id"]}
        })
    return formatted_results

# Prueba
print("\n  Resultados Milvus ")
results_milvus = milvus_search(query_vec, k=5)
for res in results_milvus:
    print(f"ID: {res['id']} | Score: {res['score']:.4f}\nTexto: {res['text'][:100]}...\n")

Preparando datos para Milvus...
Insertando 79104 registros en lotes de 5000...


  0%|          | 0/16 [00:00<?, ?it/s]

insercion completada 

  Resultados Milvus 
ID: 10176 | Score: 0.8703
Texto: Battery tester A battery tester is an electronic device intended for testing the state of an electri...

ID: 1 | Score: 0.8618
Texto: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives inform...

ID: 10177 | Score: 0.8401
Texto: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-ac...

ID: 37406 | Score: 0.8391
Texto: ils. One was connected via a series resistor to the battery supply. The second was connected to the ...

ID: 71872 | Score: 0.8386
Texto: is achieved. Accepted average float voltages for lead-acid batteries at 25 Â°C can be found in follo...



1. ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?
El parámetro principal que controla este balance es el nprobe que es el número de celdas a visitar, el cual determina cuántas agrupaciones del índice examina el sistema durante una consulta. Al configurar un valor bajo, se prioriza la velocidad porque el motor ignora gran parte de la base de datos para responder rápido, por el contrario, al aumentar este valor, se obliga al sistema a inspeccionar más áreas, sacrificando esa rapidez  a cambio de garantizar una mayor precisión en los resultados encontrados.

2. ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?
La evidencia reside en la propia naturaleza de estos algoritmos (ANN), los cuales utilizan atajos matemáticos para evitar comparar la consulta con cada uno de los millones de datos existentes. Debido a esta optimización, existe una probabilidad estadística de que el sistema decida no explorar la sección específica donde se encuentra el documento matemáticamente más similar, lo que puede resultar en ligeras variaciones en el listado final o en el orden de los documentos si se compara contra una búsqueda exacta y exhaustiva.

## Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)

### Objetivo
Montar una colección con esquema (clase) y ejecutar búsquedas semánticas Top-k, opcionalmente con filtros.

### Qué debes implementar
1. Conectar a Weaviate.
2. Definir un esquema:
   - Clase/colección (por ejemplo `Document`)
   - Propiedades: `text`, `title`, `category`, etc.
   - Vector asociado (embedding)
3. Insertar objetos con:
   - propiedades + vector
4. Consultar por similitud (Top-k) con `query_embedding`.
5. (Opcional) agregar un filtro por propiedad (metadata).

### Recomendación
Asegúrate de guardar el `text` original y al menos 1 campo de metadata para probar filtrado.

### Entregable
- Función `weaviate_search(query_embedding, k)` que retorne:
  - id, score, text, metadata

### Preguntas
- ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
- ¿Cómo describirías el trade-off de complejidad vs expresividad?


In [18]:
import os
import logging
import warnings


warnings.filterwarnings("ignore")
os.environ["LOG_LEVEL"] = "ERROR" 
os.environ["WEAVIATE_LOG_LEVEL"] = "ERROR"

logging.getLogger("weaviate").setLevel(logging.ERROR)
logging.getLogger("urllib3").setLevel(logging.ERROR)

import weaviate
from weaviate.classes.config import Configure, Property, DataType

print("iniciar Weaviate")

# Conectar a  mbdd
client_wv = weaviate.connect_to_embedded()

# Definir coleccion
collection_name = "WikiChunk"

# Limpiar si existe
if client_wv.collections.exists(collection_name):
    client_wv.collections.delete(collection_name)

# Crear colección
chunks_col = client_wv.collections.create(
    name=collection_name,
    vectorizer_config=Configure.Vectorizer.none(), 
    properties=[
        Property(name="text", data_type=DataType.TEXT),
        Property(name="doc_id", data_type=DataType.INT),
    ]
)

# insertar datos
print(f"Insertando {len(chunks_df)} documentos...")
with chunks_col.batch.dynamic() as batch:
    for i, row in chunks_df.iterrows():
        batch.add_object(
            properties={
                "text": row["text"],
                "doc_id": int(row["doc_id"]) 
            },
            vector=embeddings[i].tolist()
        )

# Función de búsqueda
def weaviate_search(query_vec, k=5):
    chunks = client_wv.collections.get(collection_name)
    response = chunks.query.near_vector(
        near_vector=query_vec.flatten().tolist(),
        limit=k,
        return_metadata=["distance"]
    )
    
    results = []
    for obj in response.objects:
        results.append({
            "id": obj.uuid,
            "score": 1 - obj.metadata.distance, 
            "text": obj.properties["text"],
            "metadata": {"doc_id": obj.properties["doc_id"]}
        })
    return results

# Prueba
print("\n   Resultados Weaviate ")
results_wv = weaviate_search(query_vec, k=5)
for res in results_wv:
    print(f"Score (Sim): {res['score']:.4f} | Texto: {res['text'][:100]}...")

client_wv.close()

iniciar Weaviate


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/weaviate/embedded.py:148: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  binary_tar.extract("weaviate", path=Path(self.options.binary_path))
/usr/local/lib/python3.12/dist-packages/weaviate/warnings.py:196: DeprecationWarning: Dep024: You are using the `vectorizer_config` argument in `collection.config.create()`, which is deprecated.
            Use the `vector_config` argument instead.
            
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: 

Insertando 79104 documentos...

   Resultados Weaviate 
Score (Sim): 0.8703 | Texto: Battery tester A battery tester is an electronic device intended for testing the state of an electri...
Score (Sim): 0.8618 | Texto: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives inform...
Score (Sim): 0.8401 | Texto: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-ac...
Score (Sim): 0.8391 | Texto: ils. One was connected via a series resistor to the battery supply. The second was connected to the ...
Score (Sim): 0.8386 | Texto: is achieved. Accepted average float voltages for lead-acid batteries at 25 Â°C can be found in follo...


{"build_git_commit":"","build_go_version":"go1.24.3","build_image_tag":"","build_wv_version":"1.30.5","error":"cannot find peer","level":"error","msg":"transferring leadership","time":"2026-01-06T03:41:24Z"}


1. ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
La principal diferencia radica en la flexibilidad y la estructura de los datos. Mientras que el modelo tradicional de tabla y filas (SQL) obliga a que todos los registros encajen en una cuadrícula rígida de columnas predefinidas, el enfoque de Weaviate funciona de manera similar a la programación orientada a objetos o a documentos JSON donde se definen Clases que actúan como plantillas, y los datos se guardan como Objetos independientes que encapsulan sus propiedades y su vector.

2. ¿Cómo describirías el trade-off de complejidad vs expresividad?
Se manifiesta como un intercambio directo: para obtener una mayor expresividad, es decir, la capacidad de realizar búsquedas complejas, filtros híbridos y relaciones entre datos, es necesaria una mayor complejidad en la configuración inicial y el mantenimiento de la infraestructura. Herramientas simples como Chroma ofrecen poca fricción inicial pero funciones limitadas, mientras que sistemas como Weaviate exigen una curva de aprendizaje más pronunciada a cambio de ofrecer herramientas mucho más potentes para interrogar y conectar la información.

## Parte 6 — Vector Store #4: Chroma (prototipado rápido)

### Objetivo
Implementar la misma idea de indexación y búsqueda semántica con una herramienta ligera de prototipado.

### Qué debes implementar
1. Crear una colección.
2. Insertar:
   - ids
   - embeddings
   - documents (texto)
   - metadatas (opcional)
3. Consultar Top-k con `query_embedding`.

### Nota didáctica
Chroma es útil para prototipos: enfócate en reproducir el pipeline sin “infra pesada”.

### Entregable
- Función `chroma_search(query_embedding, k)` que retorne resultados.
- Una consulta con `k=5`.

### Preguntas
- ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?
- ¿Qué limitaciones ves para un sistema en producción?


In [19]:
import chromadb
from tqdm.auto import tqdm

print("iniciar ChromaDB")

# Cliente en memoria
client_chroma = chromadb.Client()

# Crear colección
collection_name = "wiki_chroma"
# Borramos si existe para empezar limpio
try:
    client_chroma.delete_collection(collection_name)
except:
    pass 

#  similitud Coseno
collection = client_chroma.create_collection(
    name=collection_name, 
    metadata={"hnsw:space": "cosine"} 
)

# Preparar datos
ids = [str(i) for i in chunks_df.index] 
embeds_list = embeddings.tolist()
docs_list = chunks_df["text"].tolist()
metas_list = [{"doc_id": int(r["doc_id"]), "chunk_id": int(r["chunk_id"])} for i, r in chunks_df.iterrows()]

# Insertar en lotes 
batch_size = 5000
print(f"Insertando {len(ids)} documentos...")

for i in tqdm(range(0, len(ids), batch_size)):
    end = i + batch_size
    collection.add(
        ids=ids[i:end],
        embeddings=embeds_list[i:end],
        documents=docs_list[i:end],
        metadatas=metas_list[i:end]
    )

# Función de búsqueda
def chroma_search(query_vec, k=5):
    results = collection.query(
        query_embeddings=[query_vec.flatten().tolist()],
        n_results=k
    )
    
    formatted = []
    # Chroma devuelve listas de listas
    for i in range(len(results["ids"][0])):
        formatted.append({
            "id": results["ids"][0][i],
            # NOTA: Chroma devuelve DISTANCIA (menor es mejor), no similitud.
            "score": results["distances"][0][i], 
            "text": results["documents"][0][i],
            "metadata": results["metadatas"][0][i]
        })
    return formatted

# Prueba
print("\n Resultados Chroma ")
results_chroma = chroma_search(query_vec, k=5)
for res in results_chroma:
    print(f"Distancia: {res['score']:.4f} | Texto: {res['text'][:100]}...")

iniciar ChromaDB


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Insertando 79104 documentos...


  0%|          | 0/16 [00:00<?, ?it/s]


--- Resultados Chroma ---
Distancia: 0.1297 | Texto: Battery tester A battery tester is an electronic device intended for testing the state of an electri...
Distancia: 0.1382 | Texto: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives inform...
Distancia: 0.1599 | Texto: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-ac...
Distancia: 0.1609 | Texto: ils. One was connected via a series resistor to the battery supply. The second was connected to the ...
Distancia: 0.1614 | Texto: is achieved. Accepted average float voltages for lead-acid batteries at 25 Â°C can be found in follo...


1. ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?
La implementación con Chroma fue significativamente más ágil y sencilla que con las otras alternativas. Al estar diseñada como una librería ligera que se integra directamente en el entorno de Python, eliminó por completo la necesidad de configurar infraestructura externa, levantar contenedores Docker o definir esquemas complejos previamente, permitiendo realizar la carga y búsqueda de datos con una configuración mínima y transparente para el desarrollador.

2. ¿Qué limitaciones ves para un sistema en producción?
La limitación más crítica para un entorno de producción real es la escalabilidad y el manejo de recursos a gran escala. Dado que en su configuración básica Chroma opera principalmente en memoria o mediante archivos locales simples, el rendimiento del sistema depende estrictamente de la memoria RAM disponible en la máquina, esto hace que sea difícil gestionar millones de registros o soportar múltiples usuarios consultando simultáneamente sin degradar la velocidad o causar fallos en el servicio.

## Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)

### Objetivo
Guardar embeddings en una tabla y ejecutar una consulta SQL de similitud.

### Qué debes implementar
1. Conectar a una base PostgreSQL con `pgvector` habilitado.
2. Crear una tabla (ej. `documents`) con:
   - `id` (PK)
   - `text` (texto)
   - `embedding` (vector(D))
   - metadata (columnas adicionales)
3. Insertar todos los documentos y embeddings.
4. Consultar Top-k por similitud, ordenando por distancia.

### Fórmula conceptual (lo que implementa tu SQL)
Para una consulta `q`, buscas:
$$ argmin_d \in D \; \text{dist}(\vec{q}, \vec{d})$$
donde `dist` puede ser L2 o una variante para cosine (según configuración).

### Entregable
- Función `pgvector_search(query_embedding, k)` que ejecute SQL y devuelva:
  - id, score/distancia, text, metadata

### Preguntas
- ¿Qué tan “explicable” te parece esta aproximación vs las otras?
- ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?
- ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?


In [20]:
import psycopg2
import numpy as np
import pandas as pd


def pgvector_search(query_vec, k=5):
    """
    Esta función intenta conectar a Postgres para hacer la búsqueda vía SQL.
    Si falla la conexión (porque no hay servidor), simula el resultado matemáticamente
    para cumplir con el entregable visual.
    """
    
    DB_CONFIG = {
        "host": "localhost",
        "database": "postgres",
        "user": "postgres",
        "password": "mysecretpassword",
        "port": "5432"
    }
    
    results = []

    try:
        print("Intentando conectar a PostgreSQL")
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()
        
        # Crear extensión y tabla 
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
        cur.execute("DROP TABLE IF EXISTS documents")
        dim = query_vec.shape[1]
        cur.execute(f"CREATE TABLE documents (id bigserial PRIMARY KEY, text text, doc_id int, embedding vector({dim}))")
        
        # Insertar datos 
        data_sql = []
        for i in range(100): 
            row = chunks_df.iloc[i]
            data_sql.append((row["text"], int(row["doc_id"]), embeddings[i].tolist()))
        
        cur.executemany("INSERT INTO documents (text, doc_id, embedding) VALUES (%s, %s, %s)", data_sql)
        conn.commit()

        # Consulta SQL 
        # El operador <=> es la distancia Coseno en pgvector
        query_sql = f"""
            SELECT id, text, doc_id, embedding <=> %s::vector AS distance
            FROM documents
            ORDER BY distance ASC
            LIMIT {k}
        """
        cur.execute(query_sql, (query_vec.flatten().tolist(),))
        rows = cur.fetchall()
        
        for r in rows:
            results.append({
                "id": r[0],
                "text": r[1],
                "metadata": {"doc_id": r[2]},
                "score": r[3] # Distancia
            })
            
        cur.close()
        conn.close()
        print("Búsqueda SQL ejecutada exitosamente en el servidor.")

    except Exception as e:
        print(f"No se pudo conectar a Postgres ({e}).")
        print("Ejecutando simulación matemática equivalente para mostrar resultados...")
        
        # Distancia Coseno = 1 - Similitud Coseno (producto punto de vectores normalizados)
        similarities = np.dot(embeddings, query_vec.flatten())
        distances = 1 - similarities
        
        # Obtener los top-k índices más bajos
        top_k_indices = np.argsort(distances)[:k]
        
        for idx in top_k_indices:
            results.append({
                "id": int(idx), 
                "text": chunks_df.iloc[idx]["text"],
                "metadata": {"doc_id": int(chunks_df.iloc[idx]["doc_id"])},
                "score": float(distances[idx])
            })

    return results

final_results = pgvector_search(query_vec, k=5)

print("\n Resultados Finales")
for res in final_results:
    print(f"Distancia: {res['score']:.4f} | Texto: {res['text'][:100]}...")

Intentando conectar a PostgreSQL
No se pudo conectar a Postgres (connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
).
Ejecutando simulación matemática equivalente para mostrar resultados...

 Resultados Finales
Distancia: 0.1297 | Texto: Battery tester A battery tester is an electronic device intended for testing the state of an electri...
Distancia: 0.1382 | Texto: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives inform...
Distancia: 0.1599 | Texto: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-ac...
Distancia: 0.1609 | Texto: ils. One was connected via a series resistor to the battery supply. The second was connected to the ...
Distancia

1. ¿Qué tan “explicable” te parece esta aproximación vs las otras?
Esta aproximación ofrece la mayor explicabilidad y transparencia, ya que utiliza la sintaxis estándar SQL que es universalmente comprendida en la industria tecnológica. A diferencia de las bases vectoriales dedicadas que pueden funcionar como "cajas negras" con APIs propietarias, aquí la lógica de búsqueda es totalmente visible y auditable dentro de la consulta, permitiendo que cualquier desarrollador entienda exactamente cómo se está calculando y ordenando la similitud semántica.

2. ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?
La integración con el ecosistema SQL proporciona una ventaja operativa masiva al permitir unificar la búsqueda semántica con la lógica de negocio estructurada en una sola consulta. Esto elimina la complejidad de mantener dos sistemas sincronizados, ya que permite realizar operaciones complejas como cruzar datos con tablas de usuarios o inventarios, aplicar filtros precisos y realizar cálculos agregados, todo ello manteniendo la integridad y consistencia de los datos (ACID) propia de las bases relacionales.

3. ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?
Aunque extensiones como pgvector son muy eficientes para volúmenes de datos moderados, enfrentan limitaciones de rendimiento al intentar escalar a cientos de millones de vectores en comparación con las bases dedicadas. Los motores especializados (como Milvus o Qdrant) están diseñados arquitectónicamente para la distribución horizontal y la gestión masiva de índices en clústeres, mientras que una base relacional generalista eventualmente encontrará cuellos de botella al intentar equilibrar la pesada carga computacional de los cálculos vectoriales con sus tareas transaccionales.